In [ ]:
import json
import os.path
from transformers import AutoTokenizer, AutoModelWithLMHead

In [ ]:
model_checkpoint = "stjiris/t5-portuguese-legal-summarization"
t5_model = AutoModelWithLMHead.from_pretrained(model_checkpoint)

'''
Em caso de erro no carregamento do tokenizer abaixo, faça a instalação do transformers e sentencepiece convergentes e reinicie o ambiente de execução.

pip install datsets transformers[sentencepiece]
pip install sentencepiece
'''
t5_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# cuda device t5_model
t5_model.to('cuda:0')


In [4]:
def generate_summarization_jurisprudencia(jurisprudencia):
    jurisprudencia = jurisprudencia.replace("&nbsp;", "")
    inputs = t5_tokenizer.encode("summarize: " + jurisprudencia,
                    return_tensors='pt',
                    max_length=1000,
                    truncation=True).to('cuda:0')
    summary_ids = t5_model.generate(inputs, max_length=400, min_length=100,
        length_penalty=5., num_beams=2)

    output = t5_tokenizer.decode(summary_ids[0])
    return output.replace("summarize: ", "")

In [ ]:
# arquivo abaixo gerado em https://github.com/jonas-elias/jurisprudencia-sc-scraping
file = open(file='./export_dataset.json')

file_content = json.load(file)
texto_resumido = []
texto_temporario = []

# a cada 'x' id`s, um novo arquivo é gerado visando evitar problemas com sessões de processamento (armazenamento gradual)
quantidade_arquivo_id = 100

for content in file_content:
    texto_temporario.append({'id': content['id'], 'texto': content['texto']})
    if content['id'] % quantidade_arquivo_id == 0:
        if os.path.exists('./summarization-dataset_' + str(content['id']) + '.json') == False:
            for texto in texto_temporario:
                output = generate_summarization_jurisprudencia(texto['texto'])
                texto_resumido.append({'id': texto['id'], 'texto': output})
            with open('./summarization-dataset_' + str(content['id']) + '.json', 'w') as f:
                json.dump(texto_resumido, f)
        texto_resumido = []
        texto_temporario = []